In [1]:
# Notebook for making the figures/data for the article in Ingeniøren, mid november 2021
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [3]:
latestsubdir = list(os.walk(path_data))[0][1][-1]
latestdir = path_data + latestsubdir

dfCase = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
dfCase = dfCase.iloc[:-2]
dfCase['NewPositive'] = pd.to_numeric(dfCase['NewPositive'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['Tested'] = pd.to_numeric(dfCase['Tested'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['PosPct'] = pd.to_numeric(dfCase['PosPct'].astype(str).apply(lambda x: x.replace(',','.')))
dfCase['Date'] =  pd.to_datetime(dfCase.Date,format='%Y-%m-%d')
curDates = dfCase['Date']
dfCase.tail()


,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
650,2021-11-07,2583,106.688,2.4,6.289,112977,43.808.180
651,2021-11-08,3327,142.310,2.3,9.094,151404,43.959.584
652,2021-11-09,3541,158.737,2.2,10.113,168850,44.128.434
653,2021-11-10,3474,145.735,2.4,9.560,155295,44.283.729
654,2021-11-11,704,34.123,2.1,2.395,36518,44.320.247


In [9]:
# All in one big figure
# curDataToUse = dfCase.NewPositive
# curDates = dfCase.Date.values
curDataToUse = dfCase.NewPositive.iloc[:-1]
curDates = dfCase.Date.values[:-1]

dailydiff = curDataToUse.diff()
weekDiff = curDataToUse.values[7:] - curDataToUse.values[:-7]


firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-12-01')


fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,sharex=True,figsize=(10,16))

# Plot data, almindelig akse
ax1.plot(curDates,curDataToUse,'k.-',markersize=7,linewidth=1,label='Data')
ax1.plot(rnTime(curDates,7),rnMean(curDataToUse,7),'b',label='7-dages rullende gennemsnit')

# Plot daglig forskel
ax2.plot([curDates[0],lastDate],[0,0],':',color='grey')
ax2.plot(curDates,dailydiff,'k.-',markersize=7,linewidth=1,label='Forskel fra dagen før')
ax2.plot(rnTime(curDates,7),rnMean(dailydiff,7),'b',label='7-dages rullende gennemsnit')

# Plot ugentlig forskel
ax3.plot([curDates[0],lastDate],[0,0],':',color='grey')
ax3.plot(curDates[7:],weekDiff,'k.-',markersize=7,linewidth=1,label='Forskel fra ugen før')
ax3.plot(rnTime(curDates[7:],7),rnMean(weekDiff,7),'b',label='7-dages rullende gennemsnit')


ax4.plot(curDates,curDataToUse,'k.-',markersize=7,linewidth=1,label='Data')
ax4.plot(rnTime(curDates,7),rnMean(curDataToUse,7),'b',label='7-dages rullende gennemsnit')

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax3.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax4.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=4000)
ax2.set_ylim([-800,800])
ax3.set_ylim([-1300,1300])

ax1.set_ylabel('Antal nye smittetilfælde')
ax2.set_ylabel('Daglig forskel')
ax3.set_ylabel('Ugentlig forskel')
ax4.set_ylabel('Antal nye smittetilfælde\n(logaritmisk akse)')

ax4.set_yscale('log')
ax4.grid(axis='y',which='minor')
ax4.set_ylim([100,10000])

ax1.set_xlim([firstDate,lastDate])

ax1.legend(loc='upper left',fontsize=12)
ax2.legend(loc='upper left',fontsize=12)
ax3.legend(loc='upper left',fontsize=12)
ax4.legend(loc='upper left',fontsize=12)

# firstDate = np.datetime64('2021-08-01')
# lastDate = np.datetime64('2021-11-15')
# ax1.set_xlim([firstDate,lastDate])

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'DagTilDag_SamletFigurTilArtikel')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Split in multiple figures
# curDataToUse = dfCase.NewPositive
# curDates = dfCase.Date.values
curDataToUse = dfCase.NewPositive.iloc[:-1]
curDates = dfCase.Date.values[:-1]

dailydiff = curDataToUse.diff()
weekDiff = curDataToUse.values[7:] - curDataToUse.values[:-7]


firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-12-01')


# fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,sharex=True,figsize=(10,16))
fig1,ax1 = plt.subplots(figsize=(10,4))
fig2,ax2 = plt.subplots(figsize=(10,4))
fig3,ax3 = plt.subplots(figsize=(10,4))
fig4,ax4 = plt.subplots(figsize=(10,4))

# Plot data, almindelig akse
ax1.plot(curDates,curDataToUse,'k.-',markersize=7,linewidth=1,label='Data')
ax1.plot(rnTime(curDates,7),rnMean(curDataToUse,7),'b',label='7-dages rullende gennemsnit')

# Plot daglig forskel
ax2.plot([curDates[0],lastDate],[0,0],':',color='grey')
ax2.plot(curDates,dailydiff,'k.-',markersize=7,linewidth=1,label='Forskel fra dagen før')
ax2.plot(rnTime(curDates,7),rnMean(dailydiff,7),'b',label='7-dages rullende gennemsnit')

# Plot ugentlig forskel
ax3.plot([curDates[0],lastDate],[0,0],':',color='grey')
ax3.plot(curDates[7:],weekDiff,'k.-',markersize=7,linewidth=1,label='Forskel fra ugen før')
ax3.plot(rnTime(curDates[7:],7),rnMean(weekDiff,7),'b',label='7-dages rullende gennemsnit')


ax4.plot(curDates,curDataToUse,'k.-',markersize=7,linewidth=1,label='Data')
ax4.plot(rnTime(curDates,7),rnMean(curDataToUse,7),'b',label='7-dages rullende gennemsnit')

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax3.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax4.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=4000)
ax2.set_ylim([-800,800])
ax3.set_ylim([-1300,1300])

ax1.set_ylabel('Antal nye smittetilfælde')
ax2.set_ylabel('Daglig forskel')
ax3.set_ylabel('Ugentlig forskel')
ax4.set_ylabel('Antal nye smittetilfælde\n(logaritmisk akse)')

ax4.set_yscale('log')
ax4.grid(axis='y',which='minor')
ax4.set_ylim([100,10000])

ax1.set_xlim([firstDate,lastDate])
ax2.set_xlim([firstDate,lastDate])
ax3.set_xlim([firstDate,lastDate])
ax4.set_xlim([firstDate,lastDate])

ax1.legend(loc='upper left',fontsize=12)
ax2.legend(loc='upper left',fontsize=12)
ax3.legend(loc='lower right',fontsize=12)
ax4.legend(loc='upper left',fontsize=12)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax3.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax4.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

fig1.tight_layout()
fig2.tight_layout()
fig3.tight_layout()
fig4.tight_layout()

fig1.savefig(path_figs+'DagTilDag_FigurTilArtikel1')
fig2.savefig(path_figs+'DagTilDag_FigurTilArtikel2')
fig3.savefig(path_figs+'DagTilDag_FigurTilArtikel3')
fig4.savefig(path_figs+'DagTilDag_FigurTilArtikel4') 
# plt.savefig(path_figs+'DagTilDag_SamletFigurTilArtikel')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …